# Amadeus challenge

In [1]:
pwd

u'/home/frallain/workspace/amadeusChallenge'

In [ ]:
import sys
import csv
import datetime as dt
import sqlite3
# import numpy as np

import matplotlib.pyplot as plt
plt.style.use('ggplot')
# from matplotlib import rc
# rc('font', size=6)

import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from GeoBases import GeoBase



## First exercise : count the number of lines in Python for each file

In [ ]:
with open('bookings.csv') as f:
    for i,line in enumerate(f):
        pass
print(i+1)
with open('searches.csv') as f:
    for i,line in enumerate(f):
        pass
print(i+1)

## Second exercise : top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, 
grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. 
So to get the total number of passengers that have actually booked, 
you should sum including the negatives (that will remove the cancelled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)

In [ ]:
bookings = pd.read_csv('bookings.csv', sep='^')
bookings.columns = bookings.columns.str.strip()

bookings.pax.hist()
plt.yscale('log')

answer2 = pd.DataFrame( bookings.groupby('arr_port')['pax'].sum() )

# Just to check that there is a difference of 10 between +1 and -1 bookings
bookings[bookings.arr_port=='ZSE']['pax'].hist()

# sudo pip install geobases
# geo_a = GeoBase(data='airports', verbose=False)
geo_o = GeoBase(data='ori_por', verbose=False)
answer2['Name'] = answer2.apply(lambda x: geo_o.get(x.arr_port) )
iata2city = {}
for iata in answer2.index:
    try:
        iata2city[iata] = geo_o.get(iata)['name']
    except KeyError:
        pass
answer2['Name'] = answer2.index.map(lambda x: iata2city.get(x,'') )
answer2.sort_values(by='pax',ascending=False, inplace=True)
answer2[:10]


## Third exercise: plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona

For the arriving airport, you can use the Destination column in the searches file. 
Plot a curve for Málaga, another one for Madrid, and another one for Barcelona, 
in the same figure.

Bonus point: Solving this problem using pandas (instead of any other approach)

## Fourth exercise: match searches with bookings

For every search in the searches file, find out whether the search ended up in a booking or not 
(using the info in the bookings file). For instance, search and booking origin and destination 
should match. 
For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 
Generate a CSV file with the search data, and an additional field, containing 1 if the search 
ended up in a booking, and 0 otherwise.

Bonus point: Solving this problem using pandas (instead of any other approach)

## Bonus exercise: write a Web Service

Wrap the output of the second exercise in a web service that returns the data in JSON format (instead of printing to the standard output). The web service should accept a parameter n>0. For the top 10 airports, n is 10. For the X top airports, n is X.”
